In [ ]:
!pip install streamlit
!pip install tensorflow
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 1163.jpg to 1163.jpg
Saving 1164.jpg to 1164.jpg
Saving 1165.jpg to 1165.jpg
Saving 1525.jpg to 1525.jpg
Saving 1526.jpg to 1526.jpg
Saving 1528.jpg to 1528.jpg
Saving 1529.jpg to 1529.jpg
Saving 1530.jpg to 1530.jpg


In [ ]:
import pickle
import numpy as np
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras import Sequential
import os
from PIL import Image


In [ ]:
feature_list = np.array(pickle.load(open('vectorrepresentation.pkl', 'rb')))
filenames = pickle.load(open('filenames.pkl', 'rb'))

# Load the pre-trained ResNet50 model without the top classification layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

# Create a model pipeline
model = Sequential([
    model,
    GlobalMaxPooling2D()
])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
def feature_extraction(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    return normalized_result

def recommend(features, feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)
    distances, indices = neighbors.kneighbors([features])
    return indices


In [ ]:
print(f"Original shape of feature_list: {feature_list.shape}")

num_features = feature_list.shape[1] # Get the actual number of features

# Load the pre-trained ResNet50 model without the top classification layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

# Create a model pipeline
model = Sequential([
    model,
    GlobalMaxPooling2D()
])


def feature_extraction(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    return normalized_result

def recommend(features, feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)
    distances, indices = neighbors.kneighbors([features])
    return indices

Original shape of feature_list: (8, 100352)


In [ ]:
from google.colab import files  # If using Google Colab

# Load the pre-trained ResNet50 model without the top classification layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

# Create a model pipeline with GlobalMaxPooling2D
model = Sequential([
    model,
    GlobalMaxPooling2D()  # Output is a 2048-dimensional vector
])

# Feature extraction function
def feature_extraction(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)  # Normalize the feature vector
    return normalized_result

# Function to upload multiple images
def upload_images(num_images=2):
    print(f"Please upload {num_images} images.")
    uploaded_files = files.upload()  # Upload files in Colab
    image_paths = list(uploaded_files.keys())
    if len(image_paths) != num_images:
        raise ValueError(f"Expected {num_images} images, but got {len(image_paths)}")
    return image_paths

# Function to compare the features of two images
def compare_images(image_paths, model):
    if len(image_paths) != 2:
        raise ValueError("Exactly two images must be uploaded for comparison.")

    # Extract features for both images
    features_img1 = feature_extraction(image_paths[0], model)
    features_img2 = feature_extraction(image_paths[1], model)

    # Measure similarity using Euclidean distance or cosine similarity
    distance = np.linalg.norm(features_img1 - features_img2)  # Euclidean distance
    print(f"Euclidean Distance between the two images: {distance}")

# Upload two images
image_paths = upload_images(num_images=2)

# Compare their features
compare_images(image_paths, model)


Please upload 2 images.


Saving 1163.jpg to 1163 (1).jpg
Saving 1164.jpg to 1164 (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
Euclidean Distance between the two images: 0.5021408796310425
